<a href="https://colab.research.google.com/github/aditya-1442/Interview_AI_Assistant_LLM/blob/main/AI_Interview_Conducting_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain
!pip install OpenAI

In [6]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

# Streamlit
import streamlit as st
import requests
from bs4 import BeautifulSoup
!pip install markdownify
from markdownify import markdownify as md


In [7]:
!pip install youtube-transcript-api

In [8]:
from langchain.document_loaders import YoutubeLoader

In [9]:
import os

In [14]:
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

False

In [15]:
from google.colab import userdata

Setting up the LLM


In [17]:
OPENAI_API_KEY = userdata.get('open_api_key')
def load_LLM(openai_api_key):
    # Make sure your openai_api_key is set as an environment variable
    llm = ChatOpenAI(temperature=.7, openai_api_key=OPENAI_API_KEY, max_tokens=2000, model_name='gpt-3.5-turbo')
    return llm

### A function that will be called only if the environment's openai_api_key isn't set

In [18]:
def get_openai_api_key():
    input_text = st.text_input(label="OpenAI API Key (or set it as .env variable)",  placeholder="Ex: sk-2twmA8tfCb8un4...", key="openai_api_key_input")
    return input_text

# Here we'll pull data from a website and return it's text

In [20]:
def pull_from_website(url):
    st.write("Getting webpages...")
    # Doing a try in case it doesn't work
    try:
        response = requests.get(url)
    except:
        # In case it doesn't work
        print ("Whoops, error")
        return

    # Put your response in a beautiful soup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Get your text
    text = soup.get_text()

    # Convert your html to markdown. This reduces tokens and noise
    text = md(text)

    return text

# Pulling data from YouTube in text form

In [21]:
def get_video_transcripts(url):
    st.write("Getting YouTube Videos...")
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    documents = loader.load()
    transcript = ' '.join([doc.page_content for doc in documents])
    return transcript

# Function to change our long text about a person into documents

In [22]:
def split_text(user_information):
    # First we make our text splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=20000, chunk_overlap=2000)

    # Then we split our user information into different documents
    docs = text_splitter.create_documents([user_information])

    return docs

In [69]:
response_types = {
    'Interview Questions' : """
        Your goal is to generate interview questions that we can ask them
        Please respond with list of a few interview questions based on the topics above
    """,
    '1-Page Summary' : """
        Your goal is to generate a 1 page summary about them
        Please respond with a few short paragraphs that would prepare someone to talk to this person
    """
}

map_prompt = """You are a helpful AI bot that aids a user in research.
Below is information about a person named {persons_name}.
Information will include tweets, interview transcripts, and blog posts about {persons_name}
Use specifics from the research when possible

{response_type}

% START OF INFORMATION ABOUT {persons_name}:
{text}
% END OF INFORMATION ABOUT {persons_name}:

YOUR RESPONSE:"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text", "persons_name", "response_type"])

combine_prompt = """
You are a helpful AI bot that aids a user in research.
You will be given information about {persons_name}.
Do not make anything up, only use information which is in the person's context

{response_type}
% PERSON CONTEXT
{text}

% YOUR RESPONSE:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text", "persons_name", "response_type"])

# Start Of Streamlit page
st.set_page_config(page_title="LLM Assisted Interview Prep", page_icon=":robot:")

# Start Top Information
st.header("LLM Assisted Interview Prep")

col1, col2 = st.columns(2)

with col1:
    st.markdown("Have an interview coming up? I bet they are  YouTube or the web. This tool is meant to help you generate \
                interview questions based off of topics they've recently tweeted or talked about.\
                \n\nThis tool is powered by [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/#) [markdownify](https://pypi.org/project/markdownify/) [Tweepy](https://docs.tweepy.org/en/stable/api.html), [LangChain](https://langchain.com/) and [OpenAI](https://openai.com) and made by \
                [@JbsAditya](https://twitter.com/JbsAditya). \n\n ")



st.markdown("## :older_man: Larry The LLM Researcher")
# Output type selection by the user
output_type = st.radio(
    "Output Type:",
    ('Interview Questions', '1-Page Summary'))

In [56]:
person_name = st.text_input(label="Person's Name",  placeholder="Ex: Elad Gil", key="persons_name")
youtube_videos = st.text_input(label="YouTube URLs (Use , to seperate videos)",  placeholder="Ex: https://www.youtube.com/watch?v=c_hO_fjmMnk, https://www.youtube.com/watch?v=c_hO_fjmMnk", key="youtube_user_input")
webpages = st.text_input(label="Web Page URLs (Use , to seperate urls. Must include https://)",  placeholder="https://eladgil.com/", key="webpage_user_input")


In [57]:
st.markdown(f"### {output_type}:")

DeltaGenerator()

In [58]:
# Get URLs from a string
def parse_urls(urls_string):
    """Split the string by comma and strip leading/trailing whitespaces from each URL."""
    return [url.strip() for url in urls_string.split(',')]

In [59]:
def get_content_from_urls(urls, content_extractor):
    """Get contents from multiple urls using the provided content extractor function."""
    return "\n".join(content_extractor(url) for url in urls)


In [60]:
button_ind = st.button("*Generate Output*", type='secondary', help="Click to generate output based on information")

In [61]:
if button_ind:
    if not (youtube_videos or webpages):
        st.warning('Please provide links to parse', icon="⚠️")
        st.stop()

    if not OPENAI_API_KEY:
        st.warning('Please insert OpenAI API Key. Instructions [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key)', icon="⚠️")
        st.stop()

    if OPENAI_API_KEY == 'YourAPIKeyIfNotSet':
        # If the openai key isn't set in the env, put a text box out there
        OPENAI_API_KEY = get_openai_api_key()

In [62]:
 video_text = get_content_from_urls(parse_urls(youtube_videos), get_video_transcripts) if youtube_videos else ""
 website_data = get_content_from_urls(parse_urls(webpages), pull_from_website) if webpages else ""

 user_information = "\n".join([video_text, website_data])

In [63]:
 user_information_docs = split_text(user_information)

In [64]:
 llm = load_LLM(openai_api_key=OPENAI_API_KEY)

In [65]:
 chain = load_summarize_chain(llm,
                                 chain_type="map_reduce",
                                 map_prompt=map_prompt_template,
                                 combine_prompt=combine_prompt_template,
                                 verbose=True
                                 )

 st.write("Sending to LLM...")

In [66]:
!pip install tiktoken

In [67]:
output = chain({"input_documents": user_information_docs,
                    "persons_name": person_name,
                    "response_type" : response_types[output_type]
                    })
st.markdown(f"#### Output:")
st.write(output['output_text'])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI bot that aids a user in research.
You will be given information about .
Do not make anything up, only use information which is in the person's context


        Your goal is to generate interview questions that we can ask them
        Please respond with list of a few interview questions based on the topics above
    
% PERSON CONTEXT


% YOUR RESPONSE:


> Finished chain.

> Finished chain.


In [68]:
print(output['output_text'])

1. Can you explain the main objectives and purpose of your research project?
2. What specific research methodologies or techniques did you use in your study?
3. What were the key findings or results of your research? How do these findings contribute to the existing knowledge in the field?
4. How did you ensure the validity and reliability of your research data and findings?
5. Were there any unexpected challenges or obstacles that you encountered during your research? How did you overcome them?
6. Can you discuss any limitations or potential biases in your research design or data collection process?
7. Did your research project involve any ethical considerations? If so, how did you address them?
8. How do you plan to disseminate or publish your research findings? Do you have any specific plans for future research in this area?
9. Can you elaborate on the significance or potential impact of your research in practical or theoretical terms?
10. What are the potential applications or impli